In [ ]:
import skimage
import os

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import Progbar

In [ ]:
print(tf.__version__)

# Data Preps

In [ ]:
import logging
import time
from tqdm.notebook import tqdm

In [ ]:

User_Folder = os.path.expanduser('~')
data_path = os.path.join(User_Folder,"Desktop","archive")
print(data_path)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

In [ ]:
import shutil

In [ ]:
os.rename(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test"),os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainO"))
os.rename(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train"),os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/extras"))

In [ ]:
os.mkdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/valO"))
os.mkdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/valN"))
os.mkdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testO"))
os.mkdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testN"))
os.mkdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainN"))

624 + 250 for training 250 for validation 100 for testing

In [ ]:
source = os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/extras/")
destVal = os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/valO")
destTest = os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testO")
destTrain = os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainO")

In [ ]:
def moveData(src,dest,nos):
  for no,i in enumerate(os.listdir(src)):
    if no==nos:
      break
    shutil.move(src+i,dest+"/"+i)

In [ ]:
moveData(source,destTrain,400)
moveData(source,destVal,300)
moveData(source,destTest,200)

In [ ]:
print(len(os.listdir(destTrain)))
print(len(os.listdir(destVal)))
print(len(os.listdir(destTest)))

# Data Processing

In [ ]:
def processData(path,noisyDataPath):
  imgsPaths = os.listdir(path)
  imgsPaths = [path+"/"+i for i in imgsPaths]
  spath = noisyDataPath
  print("Processing...")
  pb_i = Progbar(len(imgsPaths))
  for no,p in enumerate(imgsPaths):
    img = image.load_img(p)
    img = image.img_to_array(img)
    img /= 255.0
    noiseImg = skimage.util.random_noise(img,mode='gaussian', clip=True,var=0.155**2)
    image.save_img(spath+"img"+str(no)+".jpeg",noiseImg)
    pb_i.add(1)
  print("Saving...")
  pb_i = Progbar(len(imgsPaths))
  for no,src in enumerate(imgsPaths):
    newName = path+"/"+"img"+str(no)+".jpeg"
    os.rename(src,newName)
    pb_i.add(1)

In [ ]:
destTrainN = os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainN/")
destValN = os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/valN/")
destTestN = os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testN/")

In [ ]:
processData(destTrain,destTrainN)

In [ ]:
processData(destVal,destValN)

In [ ]:
processData(destTest,destTestN)

In [ ]:
imgN = image.load_img(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainN/img0.jpeg"),target_size=(224,224))
imgN = image.img_to_array(imgN)
imgN /= 255.0

img1 = image.load_img(destTrain+"/"+"img0.jpeg",target_size=(224,224))
img1 = image.img_to_array(img1)
img1 /= 255.0

plt.imshow(imgN)
plt.show()
plt.imshow(img1)
plt.show()

In [ ]:
img1.shape

In [ ]:
imgN.shape

In [ ]:
def datagen(nPath, oPath, batchSize):
  noisy = []
  original = []
  i=0
  while(i!=len(nPath)):
    for iN in range(batchSize):
      if (i+iN)>=len(nPath):
        i = 2*len(nPath)
        break
      imgN = image.load_img(nPath[i+iN],target_size=(224,224))
      imgN = image.img_to_array(imgN)
      imgN /= 255.0
      noisy.append(imgN)
      imgO = image.load_img(oPath[i+iN],target_size=(224,224))
      imgO = image.img_to_array(imgO)
      imgO /= 255.0
      original.append(imgO)

    if i>=len(nPath):
      break

    yield (np.array(noisy),np.array(original))
    noisy = []
    original = []
    i += batchSize

In [ ]:
trainNoisy = os.listdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainN"))
trainNoisy = [os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainN/")+i for i in trainNoisy]

trainOriginal = os.listdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainO"))
trainOriginal = [os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/trainO/")+i for i in trainOriginal]

In [ ]:
print(trainNoisy)
print(trainOriginal)

In [ ]:
print(len(trainNoisy))
print(len(trainOriginal))

In [ ]:
valNoisy = os.listdir(os.path.join(data_path,("Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/valN")))
valNoisy = [os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/valN/")+i for i in valNoisy]

valOriginal = os.listdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/valO"))
valOriginal = [os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/valO/")+i for i in valOriginal]

In [ ]:
print(valNoisy)
print(valOriginal)

In [ ]:
print(len(valNoisy))
print(len(valOriginal))

In [ ]:
testNoisy = os.listdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testN"))
testNoisy = [os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testN/")+i for i in testNoisy]

testOriginal = os.listdir(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testO"))
testOriginal = [os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testO/")+i for i in testOriginal]

In [ ]:
print(testNoisy)
print(testOriginal)

In [ ]:
print(len(testNoisy))
print(len(testOriginal))

In [ ]:
trainDatagen = datagen(trainNoisy, trainOriginal, 100)

In [ ]:
x = 1
for imgsN,imgsO in trainDatagen:
  print(f"Batch {x}:")
  plt.imshow(imgsN[0])
  plt.show()
  print(imgsN.shape)
  plt.imshow(imgsO[0])
  plt.show()
  print(imgsO.shape)
  x+=1

In [ ]:
imgN = image.load_img(os.path.join(data_path,"Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/testN/img0.jpeg"),target_size=(224,224))
imgN = image.img_to_array(imgN)
imgN /= 255.0
imgN = np.expand_dims(imgN,axis=0)

In [ ]:
os.rename((data_path),os.path.join(User_Folder,"Desktop","Processed Images"))

In [ ]:
# DONE :)